# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [23]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()
df = df.drop(columns=["home_ownership","verification_status","issue_d", "pymnt_plan","initial_list_status","next_pymnt_d","application_type","hardship_flag","debt_settlement_flag"])

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
X = {'Current': 'low_risk'}   
df = df.replace(x)

X = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,35000.0,0.1308,797.80,125890.0,Fully Paid,30.48,0.0,0.0,14.0,0.0,...,0.0,2.0,100.0,80.0,0.0,0.0,399376.0,131350.0,32700.0,130876.0
1,10500.0,0.1719,375.35,66000.0,Current,27.24,0.0,0.0,8.0,0.0,...,0.0,3.0,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0
2,25000.0,0.2000,929.09,105000.0,Current,20.23,0.0,0.0,17.0,1.0,...,0.0,6.0,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0
3,20000.0,0.2000,529.88,56000.0,Current,24.26,0.0,0.0,8.0,0.0,...,0.0,4.0,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0
4,10000.0,0.1640,353.55,92000.0,Current,31.44,0.0,1.0,10.0,1.0,...,0.0,1.0,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70418,10000.0,0.1502,346.76,26000.0,Current,9.60,0.0,0.0,9.0,0.0,...,0.0,3.0,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0
70419,12000.0,0.2727,368.37,63000.0,Current,29.07,0.0,0.0,8.0,0.0,...,0.0,5.0,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0
70420,5000.0,0.1992,185.62,52000.0,Current,14.86,0.0,0.0,5.0,1.0,...,0.0,3.0,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0
70421,40000.0,0.0646,1225.24,520000.0,Current,9.96,0.0,1.0,21.0,0.0,...,0.0,3.0,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0


# Split the Data into Training and Testing

In [24]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape


(52817, 76)

In [25]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,70423.000000,70423.000000,70423.000000,7.042300e+04,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,...,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,70423.0,7.042300e+04,7.042300e+04,70423.000000,7.042300e+04
mean,16621.372989,0.128063,479.478688,8.843486e+04,21.745274,0.217244,0.500632,12.583787,0.125939,17549.650966,...,0.052213,2.231657,95.073013,30.515630,0.125882,0.0,2.100903e+05,6.131966e+04,29748.067038,5.572026e+04
std,10282.268052,0.048339,288.457221,1.218578e+05,20.429814,0.718822,0.760803,6.028804,0.336711,21880.270487,...,0.393946,1.907077,8.311660,33.618896,0.336648,0.0,1.928094e+05,5.735407e+04,26888.846929,5.090730e+04
min,1000.000000,0.060000,30.640000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,264.930000,5.000000e+04,13.840000,0.000000,0.000000,8.000000,0.000000,6240.500000,...,0.000000,1.000000,93.100000,0.000000,0.000000,0.0,6.700800e+04,2.645200e+04,11600.000000,2.286200e+04
50%,15000.000000,0.118000,402.890000,7.300000e+04,19.710000,0.000000,0.000000,11.000000,0.000000,12014.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.469600e+05,4.533800e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,646.740000,1.040000e+05,26.620000,0.000000,1.000000,16.000000,0.000000,21657.500000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036355e+05,7.656750e+04,39300.000000,7.250600e+04
max,40000.000000,0.308400,1676.230000,9.000000e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [26]:
# Check the balance of our target values
y.value_counts()

Current        68470
Fully Paid      1584
high_risk        347
Charged Off       22
Name: loan_status, dtype: int64

In [27]:
# Create X_train, X_test, y_train, y_test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(52817, 76)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [28]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Current': 51352,
         'Fully Paid': 51352,
         'high_risk': 51352,
         'Charged Off': 51352})

In [29]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1, max_iter=2000)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [30]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7843561526282075

In [31]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[    4,     0,     0,     1],
       [    0, 10695,    44,  6379],
       [    0,     0,   396,     0],
       [    0,    24,     1,    62]], dtype=int64)

In [32]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       1.00      0.80      1.00      0.89      0.89      0.78         5
    Current       1.00      0.62      0.95      0.77      0.77      0.57     17118
 Fully Paid       0.90      1.00      1.00      0.95      1.00      1.00       396
  high_risk       0.01      0.71      0.64      0.02      0.67      0.46        87

avg / total       0.99      0.63      0.95      0.77      0.78      0.58     17606



### SMOTE Oversampling

In [53]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, ratio=1.0).fit_resample(
    X_train, y_train
    )
from collections import Counter

Counter(y_resampled)

In [39]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.778527277561869

In [41]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[    5,     0,     0,     0],
       [  179, 10563,    87,  6289],
       [    7,     1,   388,     0],
       [    3,    37,     2,    45]], dtype=int64)

In [42]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       0.03      1.00      0.99      0.05      0.99      0.99         5
    Current       1.00      0.62      0.92      0.76      0.75      0.55     17118
 Fully Paid       0.81      0.98      0.99      0.89      0.99      0.97       396
  high_risk       0.01      0.52      0.64      0.01      0.58      0.33        87

avg / total       0.99      0.62      0.92      0.76      0.76      0.56     17606



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [43]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Charged Off': 17, 'Current': 17, 'Fully Paid': 17, 'high_risk': 17})

In [44]:
# Train the Logistic Regression model using the resampled data
odel = LogisticRegression(solver='lbfgs', random_state=1, max_iter=2000)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6739340127040008

In [46]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   5,    0,    0,    0],
       [3440, 6855,   49, 6774],
       [  84,    8,  299,    5],
       [  14,   24,    2,   47]], dtype=int64)

In [47]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       0.00      1.00      0.80      0.00      0.89      0.82         5
    Current       1.00      0.40      0.93      0.57      0.61      0.35     17118
 Fully Paid       0.85      0.76      1.00      0.80      0.87      0.73       396
  high_risk       0.01      0.54      0.61      0.01      0.58      0.33        87

avg / total       0.99      0.41      0.93      0.57      0.62      0.36     17606



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [48]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Charged Off': 51352,
         'Current': 44618,
         'Fully Paid': 51278,
         'high_risk': 51340})

In [49]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1, max_iter=2000)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [50]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8237351487242709

In [51]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   5,    0,    0,    0],
       [   0, 9771,   52, 7295],
       [   0,    0,  396,    0],
       [   0,   22,    2,   63]], dtype=int64)

In [52]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       1.00      1.00      1.00      1.00      1.00      1.00         5
    Current       1.00      0.57      0.95      0.73      0.74      0.52     17118
 Fully Paid       0.88      1.00      1.00      0.94      1.00      1.00       396
  high_risk       0.01      0.72      0.58      0.02      0.65      0.43        87

avg / total       0.99      0.58      0.95      0.73      0.74      0.53     17606

